# Distributed PM Simulation

JAX supports distributing array computations across multiple devices (GPUs/TPUs) using its sharding API. The `fwd_model_tools` library accepts an optional `sharding` parameter in `gaussian_initial_conditions` that partitions the initial field across devices. All downstream operations (`lpt`, `nbody`, painting) automatically respect this sharding.

This notebook demonstrates the distributed workflow using fake CPU devices for testing. On a multi-GPU machine, replace the `XLA_FLAGS` setup with your actual device mesh.

Make sure that you installed rich

`pip install rich`

## Setup

The `XLA_FLAGS` environment variable must be set **before** importing JAX to create fake devices on CPU.

In [1]:
import os

os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '0.97'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


import jax
import jax.numpy as jnp
import jax_cosmo as jc
import matplotlib.pyplot as plt
from jax.sharding import Mesh, NamedSharding, PartitionSpec as P , AxisType

import fwd_model_tools as ffi

print(f"Number of devices: {jax.device_count()}")
print(f"Devices {jax.devices()}")
jax.print_environment_info()

jax.config.update('jax_enable_x64' , False)

/lustre/fswork/projects/rech/tkc/commun/venv/pm/lib/python3.11/site-packages/jax/_src/compiler.py:783: UserWarning: Error reading persistent compilation cache entry for 'jit_convert_element_type': PermissionError: [Errno 13] Permission denied: '/jax_cache'
  warnings.warn(


Number of devices: 4
Devices [CudaDevice(id=0), CudaDevice(id=1), CudaDevice(id=2), CudaDevice(id=3)]
jax:    0.9.0.1
jaxlib: 0.9.0.1
numpy:  2.4.2
python: 3.11.5 | packaged by conda-forge | (main, Aug 27 2023, 03:34:09) [GCC 12.3.0]
device info: NVIDIA H100 80GB HBM3-4, 4 local devices"
process_count: 1
platform: uname_result(system='Linux', node='jzxh015', release='5.14.0-570.58.1.el9_6.x86_64', version='#1 SMP PREEMPT_DYNAMIC Tue Oct 21 04:15:07 EDT 2025', machine='x86_64')
JAX_PERSISTENT_CACHE_ENABLE_XLA_CACHES=xla_gpu_per_fusion_autotune_cache_dir
JAX_PERSISTENT_CACHE_MIN_ENTRY_SIZE_BYTES=-1
JAX_COMPILATION_CACHE_DIR=/jax_cache
JAX_PERSISTENT_CACHE_MIN_COMPILE_TIME_SECS=0
XLA_PYTHON_CLIENT_MEM_FRACTION=0.97

$ nvidia-smi
Wed Feb 18 11:56:15 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.126.16             Driver Version: 580.126.16     CUDA Version: 13.0     |
+-----------------------------------------+-----

## Create Device Mesh and Sharding

A `Mesh` maps physical devices to named axes. The `PartitionSpec` then describes how array dimensions map to mesh axes. The `jaxpm` distributed backend expects a **2D mesh** so that the first two spatial dimensions of the 3D field are each partitioned across one mesh axis.

In [2]:
import numpy as np

mesh = jax.make_mesh((4 , 1) , ('x', 'y') , axis_types=(AxisType.Auto , AxisType.Auto))
sharding = NamedSharding(mesh, P('x', 'y'))

print(f"Mesh shape: {mesh.shape}")
print(f"Partition spec: {sharding.spec}")

Mesh shape: OrderedDict([('x', 4), ('y', 1)])
Partition spec: PartitionSpec('x', 'y')


## Distributed Initial Conditions

Pass the `sharding` parameter to `gaussian_initial_conditions`. The returned `DensityField` stores the sharding info and all subsequent operations will respect it.

In [3]:
key = jax.random.PRNGKey(0)
mesh_size = (1200 , 1200 , 1200)
halo_size = (100 , 100)
box_size = (1000., 1000., 1000.)
nside = 512
flatsky = (512 , 512)
field_size = (10. , 10.)
cosmo = jc.Planck18()

initial_field = ffi.gaussian_initial_conditions(
    key, mesh_size, box_size,
    cosmo=cosmo,
    nside=nside,
    flatsky_npix=flatsky,
    field_size=field_size,
    sharding=sharding,
    halo_size=halo_size
)

print(f"Initial field shape: {initial_field.array.shape}")
print(f"Sharding: {initial_field.array.sharding}")

Initial field shape: (1200, 1200, 1200)
Sharding: NamedSharding(mesh=Mesh('x': 4, 'y': 1, axis_types=(Auto, Auto)), spec=PartitionSpec('x',), memory_kind=device)


## Run Distributed PM Pipeline

The pipeline is identical to single-device usage. JAX handles communication (halo exchanges, all-reduces) automatically based on the sharding.

In [4]:
# LPT initialization
dx, p = ffi.lpt(cosmo, initial_field, ts=0.1, order=1)

print(f"Displacement sharding: {dx.array.sharding}")
print(f"Momentum sharding: {p.array.sharding}")
jax.debug.visualize_array_sharding(dx.array[... , 0 , 0]) 

Displacement sharding: NamedSharding(mesh=Mesh('x': 4, 'y': 1, axis_types=(Auto, Auto)), spec=PartitionSpec('x',), memory_kind=device)
Momentum sharding: NamedSharding(mesh=Mesh('x': 4, 'y': 1, axis_types=(Auto, Auto)), spec=PartitionSpec('x',), memory_kind=device)


                         
          GPU 0          
                         
                         
          GPU 1          
                         
                         
          GPU 2          
                         
                         
          GPU 3          
                         

In [5]:
# N-body integration
solver = ffi.ReversibleDoubleKickDrift(
    interp_kernel=ffi.NoInterp(painting=ffi.PaintingOptions(target="flat")),
)

densities = ffi.nbody(
    cosmo, dx, p,
    t1=1.0, dt0=0.05,
    nb_shells=4,
    solver=solver,
).block_until_ready()

print(f"Lightcone type: {type(densities).__name__}")
print(f"Lightcone shape: {densities.shape}")
print(f"Density sharding: {densities.array.sharding}")
jax.debug.visualize_array_sharding(densities.array[0 , ...])

Lightcone type: FlatDensity
Lightcone shape: (4, 512, 512)
Density sharding: NamedSharding(mesh=Mesh('x': 4, 'y': 1, axis_types=(Auto, Auto)), spec=PartitionSpec(None, 'x'), memory_kind=device)


                         
          GPU 0          
                         
                         
          GPU 1          
                         
                         
          GPU 2          
                         
                         
          GPU 3          
                         

In [6]:
del densities
# N-body integration
solver = ffi.ReversibleDoubleKickDrift(
    interp_kernel=ffi.NoInterp(painting=ffi.PaintingOptions(target="spherical", scheme="bilinear")),
)

lightcone = ffi.nbody(
    cosmo, dx, p,
    t1=1.0, dt0=0.05,
    nb_shells=4,
    solver=solver,
)

print(f"Lightcone type: {type(lightcone).__name__}")
print(f"Lightcone shape: {lightcone.shape}")
jax.debug.visualize_array_sharding(lightcone.array[0 , ...])

Lightcone type: SphericalDensity
Lightcone shape: (4, 3145728)


  GPU 0    GPU 1    GPU 2    GPU 3  
                                    

In [7]:
del lightcone
# N-body integration
solver = ffi.ReversibleDoubleKickDrift(
    interp_kernel=ffi.NoInterp(painting=ffi.PaintingOptions(target="spherical", scheme="ngp")),
)

lightcone = ffi.nbody(
    cosmo, dx, p,
    t1=1.0, dt0=0.05,
    nb_shells=4,
    solver=solver,
)

print(f"Lightcone type: {type(lightcone).__name__}")
print(f"Lightcone shape: {lightcone.shape}")
jax.debug.visualize_array_sharding(lightcone.array[0 , ...])

Lightcone type: SphericalDensity
Lightcone shape: (4, 3145728)


  GPU 0    GPU 1    GPU 2    GPU 3  
                                    

In [11]:
del lightcone
# N-body integration
solver = ffi.ReversibleDoubleKickDrift(
    interp_kernel=ffi.NoInterp(painting=ffi.PaintingOptions(target="density")),
)

densities = ffi.nbody(
    cosmo, dx, p,
    t1=1.0, dt0=0.05,
    nb_shells=4,
    solver=solver,
)

print(f"Lightcone type: {type(densities).__name__}")
print(f"Lightcone shape: {densities.shape}")
print(f"Density sharding: {densities.array.sharding}")
jax.debug.visualize_array_sharding(densities.array[0 , ... , 0])

Lightcone type: DensityField
Lightcone shape: (4, 1200, 1200, 1200)
Density sharding: NamedSharding(mesh=Mesh('x': 4, 'y': 1, axis_types=(Auto, Auto)), spec=PartitionSpec(None, 'x'), memory_kind=device)


                         
          GPU 0          
                         
                         
          GPU 1          
                         
                         
          GPU 2          
                         
                         
          GPU 3          
                         